In [ ]:
import tagme, os, time

tagme.GCUBE_TOKEN = "bd8f93c8-dcec-4c5b-a348-3d3dfbcfec62-843339462"

directory = "M:\masters\semester 4\ProjectData\SyriaSpeeches" #read a certain text
FileTextList = [] #where the text inside each file is saved, correpondes to the MP name in the MPName list index (actual list from syria speeches folder)
FileNames = [] #where the file names would be saved inorder to be used later in the tag me files folder 
annotations = [] #have all the entities annotated for a certain text file
    
    
for file in os.listdir(directory): #save the text from the syria files
    filedirectorylink = os.path.join(directory,file)
    FileNames.append(file)
    fileText = open(filedirectorylink, encoding="utf8")
    FileTextList.append(fileText.read())
    
    
counter = 0 #counter to get the name of the files from the array to use as the new file name in the tag me folder    
for syriaSpeech in FileTextList: #take each speech from the FileTextList then split each line of it at every "|"
    splitFileText = syriaSpeech.split("/n") #split everyline on its own and save it in a list
#loops each line and in the file and annotate it then add the retrieved entities to the annotations list
    for line in splitFileText:
        #print("new line here: " , line)
        test_Large_text = tagme.annotate(line)
    # Print annotations with a score higher than 0.1
        for ann in test_Large_text.get_annotations(0.23):
            annotations.append(ann.entity_title)
            #print("entity : ", ann.entity_title, " , ","mention : ", ann.mention, " , ","score : ", ann.score)
    #print(annotations)

    TagMeFolderDirectory = "M:\masters\semester 4\ProjectData\SyriaSpeechesTagMe"
    TagMeFileDirectory = os.path.join(TagMeFolderDirectory, FileNames[counter])
    text_file = open(TagMeFileDirectory, "w", encoding='utf-8')    
    for a in annotations:
        inputText = a + " | "
        text_file.write(inputText)    
    text_file.close()
    print("Done file" , TagMeFileDirectory)
    #print("all annotations: ", annotations, "counter :" , counter)
    annotations.clear() #emptys the annotations array after it has been written to the file
    counter = counter + 1 
    time.sleep( 2 ) #sleeps the thread 2 seconds so it won't get banned from excessive requests
#print("all annotations: ", annotations)

In [10]:
import csv, os, shutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

csvfile = open('M:\masters\semester 4\web scraping\output files\TopicStats\Syria.csv', newline='') 
TagMeFolderDirectory = "M:\masters\semester 4\ProjectData\SyriaSpeechesTagMe"
spamreader = csv.reader(csvfile, delimiter=',')

MPNames = [] #where the MP Names are saved (actual list from syria speeches folder)
MPParty =[] #where the MP party will be saved (should have same index as MPNames)
CSVStatsMpNames = [] #where the MP Names from the CSV Statistic file are saved
CSVStatsMpParty = [] #where the MP Party from the SCV Statistics file are saved
TagMeFileTextList = [] #where the TagMe TextFiles are Saved

#function that returns the MPs party name and it takes as input the list with all mp names, list with their corresponding parties
# as well as the name of the MP we are looking for.
def getParty(CSVStatsMpNames, CSVStatsMpParty, name):
    for mpnames in CSVStatsMpNames:
        index = CSVStatsMpNames.index(mpnames)
        if(name.strip() in mpnames.strip()):
            mpparty = CSVStatsMpParty[index]
            return mpparty

for file in os.listdir(TagMeFolderDirectory): #save the text from the syria files
    filedirectorylink = os.path.join(TagMeFolderDirectory,file)
    fileText = open(filedirectorylink, encoding="utf8")
    TagMeFileTextList.append(fileText.read())
        
for fileName in os.listdir(TagMeFolderDirectory): #save the names to correspond the texts
    NameList = fileName.strip().split("_")
    MpName = ""
    if(len(NameList) >= 2):
        MpName = NameList[0] +" "+ NameList[1]
    else:
        MpName = NameList[0]
    MPNames.append(MpName)

for row in spamreader: #saving mpnames and parties from the statistics file
    mpname = row[0].strip()
    mpparty = row[1].strip()
    if(mpname not in CSVStatsMpNames):
        CSVStatsMpNames.append(mpname)
        CSVStatsMpParty.append(mpparty)
print(len(CSVStatsMpNames))
print(len(CSVStatsMpParty))

for MP in MPNames: #matchin MPnames from the folder with their parties
    index = MPNames.index(MP)
    mpparty = getParty(CSVStatsMpNames,CSVStatsMpParty, MP.strip())
    MPParty.insert(index, mpparty)
    
tfidf = TfidfVectorizer().fit_transform(TagMeFileTextList)
#cosine_similarities = linear_kernel(tfidf[0:1], tfidf).flatten()
#it gets the matrix as an input example, "[len(TagMeFileTextList)-1:len(TagMeFileTextList)]" or "[0:1]"
counter1 = 0
counter2 = 1
for file in TagMeFileTextList:
    cosine_similarities = cosine_similarity(tfidf[counter1:counter2], tfidf).flatten() 
    print(cosine_similarities)
    related_docs_indices = cosine_similarities.argsort()[:-6:-1] #first 5 related elements
    print(related_docs_indices)
    RelatedDocsParties = []
    RelatedDocMps =[]
    for doc in related_docs_indices:
        docparty = MPParty[doc]
        docMp = MPNames[doc]
        RelatedDocMps.append(docMp)
        RelatedDocsParties.append(docparty)
    
    counter1 = counter1 + 1 #counters are used to loop each file to compare it with all the others
    counter2 = counter2 + 1 #counters are used to loop each file to compare it with all the others
    
    print("related doc mps: ", RelatedDocMps)    
    print("related doc parties: ", RelatedDocsParties)
    print("related doc indices :" ,related_docs_indices)
    print(len(related_docs_indices))
    print(cosine_similarities[related_docs_indices])



116
116
[ 1.          0.60394709  0.57576742  0.0277568   0.55625158  0.55678493
  0.43427986  0.19623446  0.43023815  0.43091271  0.01918157  0.49112752
  0.53253838  0.6120939   0.29199006  0.45072054  0.51248292  0.0510872
  0.04188413  0.52476386  0.43112977  0.56993998  0.54495451  0.10708998
  0.34812609  0.02861182  0.09717292  0.39310902  0.421872    0.45165357
  0.38049954  0.39399521  0.54112187  0.54721334  0.57181275  0.04041438
  0.20996366  0.18442362  0.0637471   0.04914753  0.21157152  0.55340155
  0.05962619  0.25801291  0.56362598  0.62289209  0.04279696  0.59506399
  0.02637499  0.48723858  0.15463668  0.49987369  0.19270162  0.53592872
  0.18250223  0.56863907  0.05678732  0.50440214  0.31841015  0.30486224
  0.38060909  0.14693648  0.          0.6120939   0.54079487  0.0551545
  0.05809274  0.52751176  0.14155245  0.09170466  0.51747749  0.4980734
  0.36478874  0.53651423  0.40736549  0.443132    0.08467638  0.58132953
  0.01433976  0.41396325  0.32566842  0.284474

IndexError: list index out of range